In [1]:
!pip install evaluate accelerate

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
from transformers import TrainingArguments, Trainer, EvalPrediction
import evaluate

%matplotlib inline

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.6 MB/s eta 0:00:00


2024-08-14 22:19:04.126826: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-14 22:19:04.126921: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-14 22:19:04.250147: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# Load split data
yahoo_train = pd.read_csv("/kaggle/input/ugursayahoo-finance-news-sentences/yahoo_train.csv")
yahoo_test = pd.read_csv("/kaggle/input/ugursayahoo-finance-news-sentences/yahoo_test.csv")

In [3]:
yahoo_train.head()

,label,text
0,1,“A significant portion of the agenda will focu...
1,0,"From a year ago, it advanced 4% for a second m..."
2,1,"There are reports that Google, which is also b..."
3,2,"In addition, a downgrade of China’s outlook by..."
4,0,"But at the COP28 summit in Dubai, several init..."


In [4]:
from datasets import Dataset

ds_train = Dataset.from_pandas(yahoo_train)
ds_test = Dataset.from_pandas(yahoo_test)

In [5]:
MODEL_NAME = "google-bert/bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)


def tokenize(datapoint):
    return tokenizer(datapoint['text'], padding='max_length', truncation=True)


tokenized_train_df = ds_train.map(tokenize, batched=True)
tokenized_test_df = ds_test.map(tokenize, batched=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/17522 [00:00<?, ? examples/s]

Map:   0%|          | 0/7510 [00:00<?, ? examples/s]

In [6]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3)

training_args = TrainingArguments(
    output_dir="yahoo_bert",
    eval_strategy="epoch",
    per_device_train_batch_size=24,
    per_device_eval_batch_size=24,
    num_train_epochs=5,
    report_to="none",
)

metric = evaluate.load("accuracy")

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    y_pred = np.argmax(logits, axis=-1)
    return metric.compute(predictions=y_pred, references=labels)

In [8]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_df,
    eval_dataset=tokenized_test_df,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.385000,0.269857,0.903196
2,0.200500,0.347259,0.910519
3,0.046000,0.464147,0.917443
4,0.031300,0.520912,0.918109
5,0.005900,0.555070,0.920506


TrainOutput(global_step=3655, training_loss=0.11049416410939313, metrics={'train_runtime': 5484.5434, 'train_samples_per_second': 15.974, 'train_steps_per_second': 0.666, 'total_flos': 2.305136652715008e+16, 'train_loss': 0.11049416410939313, 'epoch': 5.0})

In [19]:
import torch
torch.cuda.empty_cache()

In [9]:
from sklearn.metrics import accuracy_score, classification_report
from transformers import EvalPrediction

# Define the evaluation metric
def eval_metrics(p: EvalPrediction):
    preds = np.argmax(p.predictions, axis=1)
    report = classification_report(p.label_ids, preds)
    print(report)
    acc = accuracy_score(p.label_ids, preds)
    return {"accuracy": acc}


# Define training arguments
eval_args = TrainingArguments(
    output_dir="./results_bert_yahoo",
    do_train=False,
    do_eval=True,
    per_device_eval_batch_size=16,
    report_to="none",
    logging_dir="./logs",
    logging_steps=10,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=eval_args,
    eval_dataset=tokenized_test_df,
    compute_metrics=eval_metrics,
)

# Run evaluation
eval_result = trainer.evaluate()

# Print results
print(eval_result)

              precision    recall  f1-score   support

           0       0.90      0.91      0.91      2204
           1       0.94      0.92      0.93      3167
           2       0.90      0.93      0.92      2139

    accuracy                           0.92      7510
   macro avg       0.92      0.92      0.92      7510
weighted avg       0.92      0.92      0.92      7510

{'eval_loss': 0.5550699234008789, 'eval_accuracy': 0.9205059920106524, 'eval_runtime': 131.3924, 'eval_samples_per_second': 57.157, 'eval_steps_per_second': 3.577}


In [10]:
import shutil
shutil.make_archive('yahoo-bert-uncased', 'zip', '/kaggle/working/yahoo_bert/checkpoint-2500')

'/kaggle/working/yahoo-bert-uncased.zip'